In [ ]:
PATIENTID = '751E1F62-93C3-8048-7F77-48415A978963'
PATIENTID = PATIENTID.upper()
print(PATIENTID)

In [ ]:
neo4j_url = "bolt://localhost:7687" 
neo4j_user = "neo4j" 
neo4j_password = "***"
neo4j_db = "poc"
SOURCE_DATASET_LOCATION = "/Users/amk/#Datasets/synthea/csv/synthea50"

In [ ]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
        .master('spark://Adityas-MacBook-Pro.local:7077')
        .appName('HealthConnect Test')
        .config('spark.ui.port', '4051')
        .config('spark.sql.repl.eagerEval.enabled', True)
        .config('spark.jars', '/Users/amk/Github/healthconnect/damg7374/code/utilities/neo4j-spark-connector.jar')
        .config("neo4j.url", neo4j_url)
        .config("neo4j.authentication.type", "basic")
        .config("neo4j.authentication.basic.username", neo4j_user)
        .config("neo4j.authentication.basic.password", neo4j_password)
        .config("neo4j.authentication.basic.password", neo4j_password)
        .config("neo4j.database", neo4j_db)
        .config('spark.executor.instances', 1)
        .config('spark.executor.cores', 1) 
        .config('spark.executor.memory', '1G') 
        .config('spark.cores.max', 1) 
        .getOrCreate())


In [ ]:
print("================")
print("patients.csv")
print("================")
SOURCE_DATASET_FILENAME = "patients.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
a = df.filter(f'upper(Id) = "{PATIENTID}"')
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")


In [ ]:
print("================")
print("encounters.csv")
print("================")

SOURCE_DATASET_FILENAME = "encounters.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
a = df.filter(f'upper(PATIENT) = "{PATIENTID}"').orderBy('Id')

a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
# Select the Encounter ID
ENCOUNTERID = 'B41D26FD-733E-7773-0115-A9E5C0451FBC'
ENCOUNTERID = ENCOUNTERID.upper()
print(ENCOUNTERID)

a = df.filter(f'upper(Id) = "{ENCOUNTERID}"')

a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

a = a = df.filter(f'upper(Id) = "{ENCOUNTERID}"').orderBy('Id').selectExpr('Id as ENCOUNTER','ORGANIZATION', 'PROVIDER', 'PAYER')
b = a.toPandas().to_dict(orient='list')
b

In [ ]:
print("================")
print("organizations.csv")
print("================")

from pyspark.sql.functions import col
c = b['ORGANIZATION']
SOURCE_DATASET_FILENAME = "organizations.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
a = df.filter(col("Id").isin(c))

a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("providers.csv")
print("================")

from pyspark.sql.functions import col
c = b['PROVIDER']
SOURCE_DATASET_FILENAME = "providers.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
a = df.filter(col("Id").isin(c))

a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("payers.csv")
print("================")

from pyspark.sql.functions import col
c = b['PAYER']
SOURCE_DATASET_FILENAME = "payers.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
a = df.filter(col("Id").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("claims.csv")
print("================")

SOURCE_DATASET_FILENAME = "claims.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
a = df.filter(f'upper(APPOINTMENTID) = "{ENCOUNTERID}"').orderBy('Id')

a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
# Select the Encounter ID
CLAIMID = '076a15e9-1da5-ada1-699d-aef0b51e534b'
CLAIMID = CLAIMID.upper()
print(CLAIMID)

x = df.filter(f'upper(Id) = "{CLAIMID}"').orderBy('Id').selectExpr('Id as CLAIM')
y = x.toPandas().to_dict(orient='list')
b.update(y)
b

In [ ]:
print("================")
print("claims_transactions.csv")
print("================")

CLAIMID = b['CLAIM'][0]
SOURCE_DATASET_FILENAME = "claims_transactions.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
a = df.filter(f'upper(CLAIMID) = upper("{CLAIMID}")').orderBy('CLAIMID', 'CHARGEID')

a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

x = df.filter(f'upper(CLAIMID) = upper("{CLAIMID}")').orderBy('CLAIMID', 'CHARGEID').selectExpr('ID as CLAIM_TRANSACTION')
y = x.toPandas().to_dict(orient='list')
b.update(y)
b


In [ ]:
print("================")
print("allergies.csv")
print("================")
SOURCE_DATASET_FILENAME = "allergies.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("careplans.csv")
print("================")
SOURCE_DATASET_FILENAME = "careplans.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("conditions.csv")
print("================")
SOURCE_DATASET_FILENAME = "conditions.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("devices.csv")
print("================")

SOURCE_DATASET_FILENAME = "devices.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("imaging_studies.csv")
print("================")

SOURCE_DATASET_FILENAME = "imaging_studies.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("immunizations.csv")
print("================")

SOURCE_DATASET_FILENAME = "immunizations.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("medications.csv")
print("================")
SOURCE_DATASET_FILENAME = "medications.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("observations.csv")
print("================")
SOURCE_DATASET_FILENAME = "observations.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("procedures.csv")
print("================")
SOURCE_DATASET_FILENAME = "procedures.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("supplies.csv")
print("================")
SOURCE_DATASET_FILENAME = "supplies.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("payer_transitions.csv")
print("================")
SOURCE_DATASET_FILENAME = "payer_transitions.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = [PATIENTID.lower()]
a = df.filter(col("PATIENT").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
b